In [1]:
import math as m
import numpy as np
import random as r
import matplotlib.pyplot as plt

In [2]:
import torch
from torch import nn
from torch import optim

In [3]:
from nflows.flows.base import Flow
from nflows.distributions.uniform import BoxUniform
from nflows.transforms.base import CompositeTransform
from nflows.transforms.autoregressive import MaskedPiecewiseRationalQuadraticAutoregressiveTransform
from nflows.transforms.autoregressive import MaskedPiecewiseQuadraticAutoregressiveTransform
from nflows.transforms.permutations import ReversePermutation
from nflows.transforms.splines.rational_quadratic import rational_quadratic_spline

In [4]:
device = torch.device("cuda:0")
#device = torch.device("cpu")

In [5]:
# Define some arbitrary probability distribution
N = (m.e-1)**2 * (1 + 8*m.pi**2) / m.e**2 / (1 + 16*m.pi**2)
def p_2d(x1_,x2_):
    if x1_ > 1. or x1_ < 0. or x2_ > 1. or x2_ < 0.:
        return 0
    return m.cos(2*m.pi*x1_)**2 *m.exp(-x1_-x2_)/N

def p_unnorm(x):
    if np.all(x > 0) and np.all(x < 1):
        return m.exp(-np.sum(x)/len(x)) * np.cos(2*m.pi*np.sum(x))**2 
    else:
        return 0

In [6]:
# Rejection sampling
def generate_data_2d(n):
    out = np.zeros((n,2))
    counter = 0
    while (counter < n):
        x = r.random()
        y = r.random()
        if (p_2d(x,y)*N > 1):
            print(p_2d(x,y)*N)   
        if (r.random() < p_2d(x,y)*N):
            out[counter][0] = x
            out[counter][1] = y
            counter += 1
    return out

def generate_data_nd(n, d):
    out = np.zeros((n, d))
    counter = 0
    while (counter < n):
        x = np.random.rand(d)
        if (r.random() < p_unnorm(x)):
            out[counter] = x
            counter += 1
    return out

In [7]:
data_size = int(1e6)
dim = 2
x_data = torch.tensor(generate_data_2d(data_size), dtype=torch.float32, device=device)

In [8]:
num_layers = 5
base_dist = BoxUniform(torch.zeros(dim), torch.ones(dim))

transforms = []
for _ in range(num_layers):
    transforms.append(ReversePermutation(features=dim))
    transforms.append(MaskedPiecewiseRationalQuadraticAutoregressiveTransform(
        features=dim, 
        hidden_features=100,
        num_bins=15,
        num_blocks=3,
        #tails="constrained"
    ))
transform = CompositeTransform(transforms)

flow = Flow(transform, base_dist).to(device)
optimizer = optim.Adam(flow.parameters())

In [ ]:
n_epochs = 100
batch_size = 10000
n_batches = m.ceil(data_size/batch_size)

for epoch in range(n_epochs):
    permutation = torch.randperm(x_data.size()[0], device=device)    

    # Loop over batches
    cum_loss = 0
    for batch in range(n_batches):
        # Set up the batch
        batch_begin = batch*batch_size
        batch_end   = min( (batch+1)*batch_size, data_size-1 )
        indices = permutation[batch_begin:batch_end]
        batch_x = x_data[indices]
        
        # Take a step
        optimizer.zero_grad()
        loss = -flow.log_prob(inputs=batch_x).mean()
        loss.backward()
        optimizer.step()

        # Compute cumulative loss
        cum_loss = (cum_loss*batch + loss.item())/(batch+1)

        print("epoch = ", epoch, "batch = ",batch+1, "/", n_batches, "loss = ", cum_loss)

epoch =  0 batch =  1 / 100 loss =  0.5751538276672363
epoch =  0 batch =  2 / 100 loss =  0.4526011794805527
epoch =  0 batch =  3 / 100 loss =  0.35642102360725403
epoch =  0 batch =  4 / 100 loss =  0.2868288066238165
epoch =  0 batch =  5 / 100 loss =  0.23166247587651015
epoch =  0 batch =  6 / 100 loss =  0.19038011056060591
epoch =  0 batch =  7 / 100 loss =  0.15620055994285004
epoch =  0 batch =  8 / 100 loss =  0.12578502937685698
epoch =  0 batch =  9 / 100 loss =  0.09643878446271022
epoch =  0 batch =  10 / 100 loss =  0.06965852743014693
epoch =  0 batch =  11 / 100 loss =  0.04321587144989859
epoch =  0 batch =  12 / 100 loss =  0.022718641208484765
epoch =  0 batch =  13 / 100 loss =  0.002547623374714297
epoch =  0 batch =  14 / 100 loss =  -0.014987185131758455
epoch =  0 batch =  15 / 100 loss =  -0.030622347754736746
epoch =  0 batch =  16 / 100 loss =  -0.04587827675277367
epoch =  0 batch =  17 / 100 loss =  -0.05912733302616021
epoch =  0 batch =  18 / 100 loss =

epoch =  1 batch =  45 / 100 loss =  -0.3860108408663007
epoch =  1 batch =  46 / 100 loss =  -0.38607200980186457
epoch =  1 batch =  47 / 100 loss =  -0.38607203897009496
epoch =  1 batch =  48 / 100 loss =  -0.38620267994701857
epoch =  1 batch =  49 / 100 loss =  -0.3862900253461331
epoch =  1 batch =  50 / 100 loss =  -0.3863795500993728
epoch =  1 batch =  51 / 100 loss =  -0.38611393407279365
epoch =  1 batch =  52 / 100 loss =  -0.3860290864339241
epoch =  1 batch =  53 / 100 loss =  -0.3860304782975394
epoch =  1 batch =  54 / 100 loss =  -0.38596574962139124
epoch =  1 batch =  55 / 100 loss =  -0.38595721938393324
epoch =  1 batch =  56 / 100 loss =  -0.3861035678003515
epoch =  1 batch =  57 / 100 loss =  -0.3861840994734512
epoch =  1 batch =  58 / 100 loss =  -0.38605237829274136
epoch =  1 batch =  59 / 100 loss =  -0.3859955890704009
epoch =  1 batch =  60 / 100 loss =  -0.38585916161537165
epoch =  1 batch =  61 / 100 loss =  -0.3861539080494739
epoch =  1 batch =  62 

epoch =  2 batch =  93 / 100 loss =  -0.3887728436659742
epoch =  2 batch =  94 / 100 loss =  -0.38875409707109987
epoch =  2 batch =  95 / 100 loss =  -0.38883995162813295
epoch =  2 batch =  96 / 100 loss =  -0.3887932834525904
epoch =  2 batch =  97 / 100 loss =  -0.38891471876311556
epoch =  2 batch =  98 / 100 loss =  -0.38889044675291806
epoch =  2 batch =  99 / 100 loss =  -0.38898420845619364
epoch =  2 batch =  100 / 100 loss =  -0.38899416178464896
epoch =  3 batch =  1 / 100 loss =  -0.381522536277771
epoch =  3 batch =  2 / 100 loss =  -0.381961852312088
epoch =  3 batch =  3 / 100 loss =  -0.38755542039871216
epoch =  3 batch =  4 / 100 loss =  -0.38791442662477493
epoch =  3 batch =  5 / 100 loss =  -0.3905823647975922
epoch =  3 batch =  6 / 100 loss =  -0.38848939041296643
epoch =  3 batch =  7 / 100 loss =  -0.3872353562286922
epoch =  3 batch =  8 / 100 loss =  -0.38740143552422523
epoch =  3 batch =  9 / 100 loss =  -0.388008713722229
epoch =  3 batch =  10 / 100 los

epoch =  4 batch =  42 / 100 loss =  -0.39069113703001124
epoch =  4 batch =  43 / 100 loss =  -0.3904916152011517
epoch =  4 batch =  44 / 100 loss =  -0.3907720981673762
epoch =  4 batch =  45 / 100 loss =  -0.39085068504015613
epoch =  4 batch =  46 / 100 loss =  -0.39094426968823315
epoch =  4 batch =  47 / 100 loss =  -0.39117414076277557
epoch =  4 batch =  48 / 100 loss =  -0.3911158597717683
epoch =  4 batch =  49 / 100 loss =  -0.3910955221069103
epoch =  4 batch =  50 / 100 loss =  -0.39098649144172676
epoch =  4 batch =  51 / 100 loss =  -0.3907086656374091
epoch =  4 batch =  52 / 100 loss =  -0.39064008799883043
epoch =  4 batch =  53 / 100 loss =  -0.3905131563825428
epoch =  4 batch =  54 / 100 loss =  -0.3906481398476495
epoch =  4 batch =  55 / 100 loss =  -0.39082171320915227
epoch =  4 batch =  56 / 100 loss =  -0.39066118427685337
epoch =  4 batch =  57 / 100 loss =  -0.39083050008405723
epoch =  4 batch =  58 / 100 loss =  -0.3907955597186911
epoch =  4 batch =  59

epoch =  5 batch =  86 / 100 loss =  -0.3906996402629587
epoch =  5 batch =  87 / 100 loss =  -0.39057807264656863
epoch =  5 batch =  88 / 100 loss =  -0.390647671439431
epoch =  5 batch =  89 / 100 loss =  -0.3906239944227627
epoch =  5 batch =  90 / 100 loss =  -0.39055388271808633
epoch =  5 batch =  91 / 100 loss =  -0.39053230658992316
epoch =  5 batch =  92 / 100 loss =  -0.3904743045568467
epoch =  5 batch =  93 / 100 loss =  -0.3904318457008691
epoch =  5 batch =  94 / 100 loss =  -0.39047455819363297
epoch =  5 batch =  95 / 100 loss =  -0.3904906548951802
epoch =  5 batch =  96 / 100 loss =  -0.3905744934454561
epoch =  5 batch =  97 / 100 loss =  -0.3906701161074885
epoch =  5 batch =  98 / 100 loss =  -0.39084812603434743
epoch =  5 batch =  99 / 100 loss =  -0.39083987775475093
epoch =  5 batch =  100 / 100 loss =  -0.3907132443785668
epoch =  6 batch =  1 / 100 loss =  -0.3827058970928192
epoch =  6 batch =  2 / 100 loss =  -0.3896953761577606
epoch =  6 batch =  3 / 100

...

epoch =  66 batch =  20 / 100 loss =  -0.3969895765185356
epoch =  66 batch =  21 / 100 loss =  -0.3959615315709795
epoch =  66 batch =  22 / 100 loss =  -0.39593063430352643
epoch =  66 batch =  23 / 100 loss =  -0.39583518064540363
epoch =  66 batch =  24 / 100 loss =  -0.39605549971262616
epoch =  66 batch =  25 / 100 loss =  -0.3961253106594086
epoch =  66 batch =  26 / 100 loss =  -0.3964910896924826
epoch =  66 batch =  27 / 100 loss =  -0.39651120812804613
epoch =  66 batch =  28 / 100 loss =  -0.3963782212563923
epoch =  66 batch =  29 / 100 loss =  -0.3962689103751347
epoch =  66 batch =  30 / 100 loss =  -0.3961508403221766
epoch =  66 batch =  31 / 100 loss =  -0.3960243732698502
epoch =  66 batch =  32 / 100 loss =  -0.39577874913811684
epoch =  66 batch =  33 / 100 loss =  -0.39569686578981805
epoch =  66 batch =  34 / 100 loss =  -0.3955513852484086
epoch =  66 batch =  35 / 100 loss =  -0.39576489414487565
epoch =  66 batch =  36 / 100 loss =  -0.39572977854145897
epoch 

epoch =  67 batch =  64 / 100 loss =  -0.3957055704668164
epoch =  67 batch =  65 / 100 loss =  -0.3957330456146827
epoch =  67 batch =  66 / 100 loss =  -0.3957640898950172
epoch =  67 batch =  67 / 100 loss =  -0.39578145208643445
epoch =  67 batch =  68 / 100 loss =  -0.39584220332257886
epoch =  67 batch =  69 / 100 loss =  -0.39577799473983655
epoch =  67 batch =  70 / 100 loss =  -0.3958029040268489
epoch =  67 batch =  71 / 100 loss =  -0.3957330479588307
epoch =  67 batch =  72 / 100 loss =  -0.39572645392682815
epoch =  67 batch =  73 / 100 loss =  -0.395669697082206
epoch =  67 batch =  74 / 100 loss =  -0.3957575015925072
epoch =  67 batch =  75 / 100 loss =  -0.3957677511374156
epoch =  67 batch =  76 / 100 loss =  -0.39579112435642044
epoch =  67 batch =  77 / 100 loss =  -0.39576371497922136
epoch =  67 batch =  78 / 100 loss =  -0.39590298747405034
epoch =  67 batch =  79 / 100 loss =  -0.3960105784331696
epoch =  67 batch =  80 / 100 loss =  -0.3959820058196783
epoch = 

epoch =  69 batch =  8 / 100 loss =  -0.3925073966383934
epoch =  69 batch =  9 / 100 loss =  -0.3934083316061232
epoch =  69 batch =  10 / 100 loss =  -0.39327520728111265
epoch =  69 batch =  11 / 100 loss =  -0.39341534809632733
epoch =  69 batch =  12 / 100 loss =  -0.3932737708091736
epoch =  69 batch =  13 / 100 loss =  -0.3935311918075268
epoch =  69 batch =  14 / 100 loss =  -0.3943320023162024
epoch =  69 batch =  15 / 100 loss =  -0.3946003794670105
epoch =  69 batch =  16 / 100 loss =  -0.3944011218845844
epoch =  69 batch =  17 / 100 loss =  -0.39380387348287244
epoch =  69 batch =  18 / 100 loss =  -0.3947535388999515
epoch =  69 batch =  19 / 100 loss =  -0.3951177251966376
epoch =  69 batch =  20 / 100 loss =  -0.39526255875825883
epoch =  69 batch =  21 / 100 loss =  -0.3948921873455956
epoch =  69 batch =  22 / 100 loss =  -0.39531962167132983
epoch =  69 batch =  23 / 100 loss =  -0.395700700905012
epoch =  69 batch =  24 / 100 loss =  -0.3953633705774943
epoch =  69 

epoch =  70 batch =  53 / 100 loss =  -0.39531079384515866
epoch =  70 batch =  54 / 100 loss =  -0.395307817392879
epoch =  70 batch =  55 / 100 loss =  -0.3954361460425636
epoch =  70 batch =  56 / 100 loss =  -0.3956435253577572
epoch =  70 batch =  57 / 100 loss =  -0.3955667651536171
epoch =  70 batch =  58 / 100 loss =  -0.395639263350388
epoch =  70 batch =  59 / 100 loss =  -0.3960062487650725
epoch =  70 batch =  60 / 100 loss =  -0.39601758519808444
epoch =  70 batch =  61 / 100 loss =  -0.39603967744796
epoch =  70 batch =  62 / 100 loss =  -0.3959165282787815
epoch =  70 batch =  63 / 100 loss =  -0.3960396046676332
epoch =  70 batch =  64 / 100 loss =  -0.3960160394199192
epoch =  70 batch =  65 / 100 loss =  -0.39587526825758124
epoch =  70 batch =  66 / 100 loss =  -0.395730892365629
epoch =  70 batch =  67 / 100 loss =  -0.3957716288851268
epoch =  70 batch =  68 / 100 loss =  -0.39569018518223475
epoch =  70 batch =  69 / 100 loss =  -0.3957820776580036
epoch =  70 bat

epoch =  71 batch =  97 / 100 loss =  -0.39598461678347635
epoch =  71 batch =  98 / 100 loss =  -0.39599981113355986
epoch =  71 batch =  99 / 100 loss =  -0.3960416922063539
epoch =  71 batch =  100 / 100 loss =  -0.39609434694051743
epoch =  72 batch =  1 / 100 loss =  -0.4027171730995178
epoch =  72 batch =  2 / 100 loss =  -0.387325644493103


In [ ]:
with torch.no_grad():
    x_flow = flow.sample(100000).cpu().numpy()

In [ ]:
plt.hist2d(x_data[:,0].cpu().numpy(), x_data[:,1].cpu().numpy(), bins=25)
plt.show()

In [ ]:
plt.hist2d(x_flow[:,0], x_flow[:,1], bins=25)
plt.show()